In [14]:
import rispy
import pandas as pd
from tqdm import tqdm
from itertools import chain

# Path to your RIS file
ris_file = 'TAS_200 records.ris'

# Read the RIS file with rispy
with open(ris_file, 'r') as file:
    records = rispy.load(file)

# Convert to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
df.head(2)


,type_of_reference,title,authors,abstract,date,year,secondary_title,doi,volume,number,alternate_title1,start_page,end_page,url,accession_number
0,JOUR,Screening for Cognitive Impairment in Older Ad...,"[Patnode, Carrie D., Perdue, Leslie A., Rossom...",OBJECTIVE: We conducted this systematic review...,2020///,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JOUR,Interventions to Prevent Falls in Community-Dw...,"[Guirguis-Blake, Janelle M., Michael, Yvonne L...",OBJECTIVE: We conducted this systematic review...,2018///,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
pico_compliance_results = []
for item in tqdm(df.values):
    text = item[3]
    
    sample_sentences = sent_tokenize(preprocess_text(str(text)))
    
    sample_vectors = vectorizer.transform(sample_sentences)
    
    sample_pred = clf.predict(sample_vectors)
    predicted_prob = clf.predict_proba(sample_vectors)

    predicted_labels = mlb.inverse_transform(sample_pred)
    
    existing_flags = set(chain.from_iterable(predicted_labels))
    
    required_flags = {'P', 'I', 'C', 'O'}
    pico_compliant = required_flags.issubset(existing_flags)
    
#     pico_compliance_results.append(pico_compliant)
    
    pico_compliance_results.append({
        "PICO_Compliant": pico_compliant,
        "P": 'P' in existing_flags,
        "I": 'I' in existing_flags,
        "C": 'C' in existing_flags,
        "O": 'O' in existing_flags
    })

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 69.83it/s]


In [19]:
df = pd.concat([df, pd.DataFrame(pico_compliance_results)], axis=1)

In [20]:
df

,type_of_reference,title,authors,abstract,date,year,secondary_title,doi,volume,number,alternate_title1,start_page,end_page,url,accession_number,PICO_Compliant,P,I,C,O
0,JOUR,Screening for Cognitive Impairment in Older Ad...,"[Patnode, Carrie D., Perdue, Leslie A., Rossom...",OBJECTIVE: We conducted this systematic review...,2020///,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True
1,JOUR,Interventions to Prevent Falls in Community-Dw...,"[Guirguis-Blake, Janelle M., Michael, Yvonne L...",OBJECTIVE: We conducted this systematic review...,2018///,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True
2,JOUR,Effect of Dancing Interventions on Depression ...,"[Prudente, Tiago Paiva, Mezaiko, Eleazar, Silv...",This systematic review and meta-analysis (PROS...,2024///,2024,"Behavioral sciences (Basel, Switzerland)",10.3390/bs14010043,14,1,"Behavioral sciences (Basel, Switzerland)",NaN,NaN,NaN,NaN,True,True,True,True,True
3,JOUR,Dose-response relationship of high-intensity t...,"[Afanador-Restrepo, Diego Fernando, Casanova-C...",BACKGROUND: The prevalence of mild cognitive i...,2024///,2024,European review of aging and physical activity...,10.1186/s11556-024-00358-3,21,1,European review of aging and physical activity...,23,NaN,NaN,NaN,True,True,True,True,True
4,JOUR,Effectiveness of Serious Games for Improving E...,"[Abd-Alrazaq, Alaa, Alhuwail, Dari, Ahmed, Arf...",BACKGROUND: Executive functions are one of the...,2022///,2022,JMIR serious games,10.2196/36123,10,3,JMIR serious games,e36123,NaN,NaN,NaN,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,JOUR,Evaluation of two healthy lifestyle and exerci...,NaN,INTERVENTION: The Healthy and Active for Life ...,2020///,2020,Evaluation of two healthy lifestyle and exerci...,NaN,NaN,NaN,Evaluation of two healthy lifestyle and exerci...,NaN,NaN,NaN,NaN,False,False,True,False,True
196,JOUR,Effects of a program combining walking and cog...,NaN,INTERVENTION: Walking has a positive influence...,2011///,2011,The influence of combined physical training an...,NaN,NaN,NaN,The influence of combined physical training an...,NaN,NaN,NaN,NaN,True,True,True,True,True
197,JOUR,Effects of a remotely supervised home exercise...,NaN,INTERVENTION: D005081 The sample will consist...,2021///,2021,Effects of a remotely supervised home-based ex...,NaN,NaN,NaN,Effects of a remotely supervised home-based ex...,NaN,NaN,NaN,NaN,True,True,True,True,True
198,JOUR,Study of Exercise on Impact of Cognitive Funct...,NaN,Cognitive impairment affects roughly 50% of pe...,2014///,2014,The Effect of Aerobic Exercise on Cognition in...,NaN,NaN,NaN,The Effect of Aerobic Exercise on Cognition in...,NaN,NaN,NaN,NaN,False,True,True,True,False


In [21]:
df.to_excel("TAS_200_with_PICO_flags.xlsx")

In [1]:
# import re

# def re_label_comparison(text):
#     text = re.sub(r"\b\|C\|\b", "|X|", text)
    
#     comparison_patterns = [
#         r"\bcompare\b",
#         r"\bcompared\b",
#         r"\bversus\b|\bvs\b",
#         r"\bcomparison\b",
#         r"\bcomparable\b",
#         r"\bcomparing\b"
#     ]
    
#     lines = text.split('\n')
#     re_labeled_lines = []
    
#     for line in lines:
#         for pattern in comparison_patterns:
#             if re.search(pattern, line, re.IGNORECASE):
#                 # Re-label as "C" (Comparison)
#                 line = re.sub(r"\b\|A\|\b", "|C|", line)
#                 line = re.sub(r"\b\|R\|\b", "|C|", line)
#                 line = re.sub(r"\b\|X\|\b", "|C|", line)
#         re_labeled_lines.append(line)
    
#     return "\n".join(re_labeled_lines)


# input_file = '.\\PICO\\data\\splitted\\PICO_train.txt'
# output_file = '.\\PICO\\data\\splitted\\PICO_train_relabelled.txt'

# with open(input_file, 'r') as f:
#     data = f.read()

# relabelled_data = re_label_comparison(data)

# with open(output_file, 'w') as f:
#     f.write(relabelled_data)

# print(f"Re-labeled dataset has been saved to {output_file}")


Re-labeled dataset has been saved to .\PICO\data\splitted\PICO_train_relabelled.txt


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
# from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Initialize the lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean and preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation.replace('.', '').replace('!', '').replace('?', '')))
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
#     tokens = [word for word in tokens if word not in stop_words]
    
    # Join tokens back to string
    return ' '.join(tokens)


In [4]:
def parse_pico_dataset(filepath):
    data = []
    with open(filepath, 'r') as file:
        for line in file:
            if '|' in line:
                name, label, text = line.strip().split('|', 2)
                
                if label in ['P', 'I', 'C', 'O']:  # Filter only PICO elements
                    clean_text = preprocess_text(text)
#                     if label == 'R':
#                         label = 'O'
                    data.append({'label': label, 'text': clean_text})
    return pd.DataFrame(data)

# Load the dataset
df = parse_pico_dataset('.\\PICO\\data\\splitted\\PICO_train_relabelled.txt')

In [5]:
df

,label,text
0,P,people crc considered inclusion trial year old...
1,P,n eligible crc survivor consented participate ...
2,P,crc survivor people cardiovascular disease cvd...
3,I,referral postsurgical crc survivor weekly cr e...
4,I,class included crc survivor people cvd .
...,...,...
90180,I,intervention based transtheoretical model beha...
90181,O,selfreported point prevalence prolonged abstin...
90182,C,analysis including followup confirmed statisti...
90183,P,fifty eye consecutive patient clinically signi...


In [6]:
len(df[df['label']=='P']), len(df[df['label']=='I']), len(df[df['label']=='C']), len(df[df['label']=='O'])

(22347, 19864, 21745, 26229)

In [7]:
# Map labels to integers for model training
df['label'] = df['label'].apply(lambda x: x if isinstance(x, list) else [str(x)])

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)


In [8]:
# Convert text data to a bag-of-words representation
vectorizer = CountVectorizer()
# vectorizer = TfidfVectorizer()

# Transform the training and validation text data
X_train = vectorizer.fit_transform(train_texts)
X_val = vectorizer.transform(val_texts)

In [9]:
# # Initialize the Naive Bayes classifier
# nb_model = MultinomialNB()

# # Train the model
# nb_model.fit(X_train, train_labels)

mlb = MultiLabelBinarizer(classes=['P', 'I', 'C', 'O'])
y_train = mlb.fit_transform(train_labels)
y_val = mlb.transform(val_labels)

In [10]:
# Initialize the OneVsRestClassifier with MultinomialNB
clf = OneVsRestClassifier(MultinomialNB())

# Train the model
clf.fit(X_train, y_train)


OneVsRestClassifier(estimator=MultinomialNB())

In [11]:
# Make predictions on the validation set
y_pred = clf.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=['Population', 'Intervention', 'Comparison', 'Outcome'], zero_division=0)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.7606164763277525
Classification Report:
              precision    recall  f1-score   support

  Population       0.85      0.83      0.84      2222
Intervention       0.78      0.76      0.77      1972
  Comparison       0.82      0.90      0.86      2206
     Outcome       0.84      0.84      0.84      2619

   micro avg       0.82      0.83      0.83      9019
   macro avg       0.82      0.83      0.83      9019
weighted avg       0.82      0.83      0.83      9019
 samples avg       0.80      0.83      0.81      9019



In [12]:
sample_text = """
OBJECTIVE|A|The aim of this study was to verify the efficacy and applicability of the Ultracision Harmonic Scalpel ( Ethicon Endo-Surgery , Cincinnati , OH ) in pharyngolaryngectomy with radical neck dissection .
DESIGN|M|This work evaluated the use of the Harmonic Scalpel in otolaryngology as a new and alternative method to overcome some complications of traditional surgery .
SETTING|M|The study was conducted with @ patients in the ENT Department of the University of Genoa ( Italy ) between January @ , @ , and December @ , @ .
METHODS|M|A prospective , randomized study was undertaken on @ pharyngolaryngectomies with radical neck dissection performed using a Harmonic Scalpel ( @ patients ) or traditional surgery ( @ patients ) .
MAIN OUTCOME MEASURES|O|The evaluation included operation time , intraoperative blood loss , postoperative seroma formation , and pattern of wound healing .
RESULTS|R|In patients treated with Harmonic Scalpel , the mean operation time was significantly shorter , the blood loss was less , laryngeal and neck drainage on the first and second postoperative days was significantly smaller , and no postoperative complications were noted .
CONCLUSIONS|C|The use of the Harmonic Scalpel during pharyngolaryngectomy and radical neck dissection led to diminished bleeding , shorter operative time , less seroma formation , and better wound healing in the postoperative period .
"""

sample_text = """
Title:
The Effect of Aerobic Exercise on Cognitive Function in Elderly Patients with Mild Cognitive Impairment

Abstract:
This study investigates the impact of a structured aerobic exercise program on cognitive function in elderly patients diagnosed with mild cognitive impairment (MCI). We conducted a randomized controlled trial to compare the cognitive outcomes of patients participating in the exercise program versus a control group receiving usual care.

Introduction:
Mild cognitive impairment (MCI) is a condition characterized by cognitive decline that is more pronounced than normal aging but not severe enough to interfere significantly with daily activities. Exercise has been proposed as a potential intervention to improve cognitive function in this population.

Methods:
Population: The study included 100 elderly patients aged 65 and above diagnosed with MCI based on clinical criteria.

Intervention: Participants in the intervention group engaged in a structured aerobic exercise program, consisting of 45-minute sessions of moderate-intensity exercise, three times per week for six months.

Comparison: The control group received usual care, which included standard medical treatment and advice on maintaining a healthy lifestyle but did not include a structured exercise program.

Outcome: The primary outcome measure was the change in cognitive function, assessed using the Mini-Mental State Examination (MMSE) score at baseline and after six months. Secondary outcomes included changes in memory, attention, and executive function, assessed using standardized neuropsychological tests.

Results:
At the end of the study period, the intervention group showed a statistically significant improvement in MMSE scores compared to the control group (mean difference: 2.5 points, 95% CI: 1.2-3.8, p < 0.001). Improvements were also observed in memory and executive function tests.

Conclusion:
Aerobic exercise has a positive effect on cognitive function in elderly patients with MCI. Regular participation in structured exercise programs may be a beneficial non-pharmacological intervention to delay cognitive decline in this population.

Keywords: mild cognitive impairment, aerobic exercise, cognitive function, elderly, randomized controlled trial

Discussion:
Our findings suggest that aerobic exercise can significantly improve cognitive function in elderly patients with MCI. This aligns with previous research indicating the benefits of physical activity on brain health. Further studies with larger sample sizes and longer follow-up periods are warranted to confirm these results and explore the mechanisms underlying the observed benefits.

References:

Smith, A., et al. (2021). The effects of physical activity on cognitive function in older adults. Journal of Aging Research, 25(3), 215-229.
Johnson, L., et al. (2020). Exercise interventions for cognitive decline: A systematic review. The Gerontologist, 60(4), 517-529.
"""


# Split the sample text into sentences
sample_sentences = sent_tokenize(preprocess_text(sample_text))

# Transform the sentences using the vectorizer
sample_vectors = vectorizer.transform(sample_sentences)

# Predict the PICO labels for each sentence
sample_pred = clf.predict(sample_vectors)
predicted_prob = clf.predict_proba(sample_vectors)

# Map the binary output to label names
predicted_labels = mlb.inverse_transform(sample_pred)

# Display the predicted PICO elements
for sentence, labels, probs in zip(sample_sentences, predicted_labels, predicted_prob):
    print(f"Sentence: {sentence.strip()}")
    print("Probabilities: ", probs)
    print(f"Predicted PICO Elements: {', '.join(labels) if labels else 'None'}\n")

# Check if the text is PICO compliant
pico_compliance = set(['P', 'I', 'C', 'O']).issubset(set([label for labels in predicted_labels for label in labels]))
print(f"Is the sample text PICO compliant? {'Yes' if pico_compliance else 'No'}")


Sentence: title effect aerobic exercise cognitive function elderly patient mild cognitive impairment abstract study investigates impact structured aerobic exercise program cognitive function elderly patient diagnosed mild cognitive impairment mci .
Probabilities:  [4.79213963e-01 2.12355894e-08 1.33700923e-04 1.01513730e-03]
Predicted PICO Elements: None

Sentence: conducted randomized controlled trial compare cognitive outcome patient participating exercise program versus control group receiving usual care .
Probabilities:  [2.40601765e-04 6.25372673e-03 5.15819286e-01 3.14433994e-09]
Predicted PICO Elements: C

Sentence: introduction mild cognitive impairment mci condition characterized cognitive decline pronounced normal aging severe enough interfere significantly daily activity .
Probabilities:  [1.32775226e-02 8.08825578e-08 8.26692910e-01 1.10937188e-03]
Predicted PICO Elements: C

Sentence: exercise proposed potential intervention improve cognitive function population .
Probabil

In [13]:
import psycopg2
from tqdm import tqdm

In [14]:
db_host = "awseb-e-mmtzduxdgy-stack-awsebrdsdatabase-a1ggrejgeign.cp5mioiwgdbp.ca-central-1.rds.amazonaws.com"
db_name = "user_db"
db_user = "bsituser"
db_password = "M4pbcMDsbm30zDV6"
db_port = "5432"

In [15]:
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)
cursor = conn.cursor()

In [16]:
cursor.execute("""
    ALTER TABLE chunks
    ADD COLUMN IF NOT EXISTS P BOOLEAN,
    ADD COLUMN IF NOT EXISTS I BOOLEAN,
    ADD COLUMN IF NOT EXISTS C BOOLEAN,
    ADD COLUMN IF NOT EXISTS O BOOLEAN;
""")
conn.commit()

In [17]:
def label_extraction(predicted_labels):
    P, I, C, O = False, False, False, False
    labels = set()
    for l0 in predicted_labels:
        for l1 in l0:
            labels.add(l1)
    if 'P' in labels:
        P = True
    if 'I' in labels:
        I = True
    if 'C' in labels:
        C = True
    if 'O' in labels:
        O = True
            
    return P, I, C, O

In [18]:
cursor.execute("SELECT id, string_value FROM chunks;")
rows = cursor.fetchall()

In [19]:
for row in tqdm(rows):
    row_id, text = row

    sentences = sent_tokenize(preprocess_text(text))
    vectors = vectorizer.transform(sentences)
    y_pred = clf.predict(vectors)
    predicted_labels = mlb.inverse_transform(y_pred)

    P, I, C, O = label_extraction(predicted_labels)

    cursor.execute("""
        UPDATE chunks
        SET P = %s, I = %s, C = %s, O = %s
        WHERE id = %s;
    """, (P, I, C, O, row_id))

conn.commit()

100%|████████████████████████████████████████████████████████████████████████████| 35617/35617 [14:19<00:00, 41.43it/s]


In [20]:
cursor.close()
conn.close()

In [226]:
# doc2vec

In [227]:
# Load your dataset and preprocess text as you did before
df = parse_pico_dataset('.\\PICO\\data\\splitted\\PICO_train.txt')

# Ensure the labels are in the correct format (list of strings)
df['label'] = df['label'].apply(lambda x: [str(x)])

# Tokenize the text and tag each sentence with a unique identifier
df['tokens'] = df['text'].apply(lambda x: word_tokenize(x))

# Prepare the TaggedDocument for Doc2Vec
tagged_data = [TaggedDocument(words=row['tokens'], tags=[str(i)]) for i, row in df.iterrows()]

# Split the data into training and validation sets
train_data, val_data = train_test_split(tagged_data, test_size=0.1, random_state=42)

train_texts = [doc.words for doc in train_data]
train_labels = [doc.tags[0] for doc in train_data]
val_texts = [doc.words for doc in val_data]
val_labels = [doc.tags[0] for doc in val_data]

In [228]:
# Initialize and train the Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=2, workers=4, epochs=40)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [231]:
# Function to get vectors from the trained Doc2Vec model
def get_vectors(doc2vec_model, docs):
    return np.array([doc2vec_model.infer_vector(doc) for doc in docs])

# Generate vectors for the training and validation sets
X_train = get_vectors(doc2vec_model, train_texts)
X_val = get_vectors(doc2vec_model, val_texts)

In [233]:
# Map the labels back to integers
label_mapping = {'P': 0, 'I': 1, 'C': 2, 'O': 3}
y_train = [label_mapping[df.loc[int(label)]['label'][0]] for label in train_labels]
y_val = [label_mapping[df.loc[int(label)]['label'][0]] for label in val_labels]

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Print the classification report
print(classification_report(y_val, y_pred, target_names=['Population', 'Intervention', 'Comparison', 'Outcome']))

              precision    recall  f1-score   support

  Population       0.62      0.59      0.60      2223
Intervention       0.59      0.46      0.51      1927
  Comparison       0.70      0.78      0.74      3715
     Outcome       0.62      0.66      0.64      2540

    accuracy                           0.65     10405
   macro avg       0.63      0.62      0.62     10405
weighted avg       0.64      0.65      0.64     10405



In [116]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Load a pre-trained BERT model for sequence classification with 5 labels
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Define a pipeline for text classification
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


C:\Users\pouri\anaconda3\envs\spacy_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\pouri\anaconda3\envs\spacy_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [117]:
def classify_pico_element(sentence):
    result = classifier(sentence)
#     print(result)
    label = result[0]['label']
    
    # Map star ratings to PICO elements
    if label == '1 star':
        return "Population"
    elif label == '2 stars':
        return "Intervention"
    elif label == '3 stars':
        return "Comparison"
    elif label == '4 stars' or label == '5 stars':
        return "Outcome"
    else:
        return "Unknown"

def extract_pico(text):
    sentences = text.split('.')
    pico_elements = {
        "Population": [],
        "Intervention": [],
        "Comparison": [],
        "Outcome": []
    }
    
    for sentence in sentences:
        element = classify_pico_element(sentence)
        if element in pico_elements:
            pico_elements[element].append(sentence.strip())

    return pico_elements

def is_pico_compliant(pico_elements):
    return all(bool(pico_elements[key]) for key in ["Population", "Intervention", "Comparison", "Outcome"])


In [118]:
sample_text = """
OBJECTIVE|A|The aim of this study was to verify the efficacy and applicability of the Ultracision Harmonic Scalpel ( Ethicon Endo-Surgery , Cincinnati , OH ) in pharyngolaryngectomy with radical neck dissection .
DESIGN|M|This work evaluated the use of the Harmonic Scalpel in otolaryngology as a new and alternative method to overcome some complications of traditional surgery .
SETTING|M|The study was conducted with @ patients in the ENT Department of the University of Genoa ( Italy ) between January @ , @ , and December @ , @ .
METHODS|M|A prospective , randomized study was undertaken on @ pharyngolaryngectomies with radical neck dissection performed using a Harmonic Scalpel ( @ patients ) or traditional surgery ( @ patients ) .
MAIN OUTCOME MEASURES|O|The evaluation included operation time , intraoperative blood loss , postoperative seroma formation , and pattern of wound healing .
RESULTS|R|In patients treated with Harmonic Scalpel , the mean operation time was significantly shorter , the blood loss was less , laryngeal and neck drainage on the first and second postoperative days was significantly smaller , and no postoperative complications were noted .
CONCLUSIONS|C|The use of the Harmonic Scalpel during pharyngolaryngectomy and radical neck dissection led to diminished bleeding , shorter operative time , less seroma formation , and better wound healing in the postoperative period .
"""

# Extract PICO elements
pico_elements = extract_pico(sample_text)

# Check if the text is PICO compliant
compliant = is_pico_compliant(pico_elements)

print(f"PICO Elements: {pico_elements}")
print(f"PICO Compliant: {compliant}")

PICO Elements: {'Population': [], 'Intervention': [], 'Comparison': ['RESULTS|R|In patients treated with Harmonic Scalpel , the mean operation time was significantly shorter , the blood loss was less , laryngeal and neck drainage on the first and second postoperative days was significantly smaller , and no postoperative complications were noted', 'CONCLUSIONS|C|The use of the Harmonic Scalpel during pharyngolaryngectomy and radical neck dissection led to diminished bleeding , shorter operative time , less seroma formation , and better wound healing in the postoperative period'], 'Outcome': ['OBJECTIVE|A|The aim of this study was to verify the efficacy and applicability of the Ultracision Harmonic Scalpel ( Ethicon Endo-Surgery , Cincinnati , OH ) in pharyngolaryngectomy with radical neck dissection', 'DESIGN|M|This work evaluated the use of the Harmonic Scalpel in otolaryngology as a new and alternative method to overcome some complications of traditional surgery', 'SETTING|M|The study

In [119]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load the BioBERT model and tokenizer
# model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
# model_name = "blaze999/Medical-NER"
model_name = "d4data/biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


In [120]:
import re

def chunk_text(text, tokenizer, chunk_size=512):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=chunk_size)
    token_chunks = [tokens.input_ids[i:i + chunk_size] for i in range(0, tokens.input_ids.size(1), chunk_size)]
    return token_chunks

def detect_pico(text):
    pico = {'Population': False, 'Intervention': False, 'Comparison': False, 'Outcome': False}
    
    # Define refined criteria patterns
    criteria_patterns = {
        'Population': [
            r'\b(older people|patients|population|subjects|participants)\b',
            r'\b(over 65 years of age|with Parkinson’s Disease|with mild cognitive impairment|at risk of dementia|diagnosed with dementia)\b',
            r'\bDSM\b',
            r'\bICD-10\b',
            r'\bNational Institute of Neurological and Communicative Disorders and Stroke\b',
            r'\bAlzheimer\'s Disease and Related Disorders Association\b',
            r'\bCERAD-K\b',
            r'\b(population|patient|cohort|sample|demographic|inclusion criteria)\b'
        ],
        'Intervention': [
            r'\b(exercise programs|treatment|intervention|therapy|procedure|medication|rehabilitation)\b',
            r'\b(aerobic|strength|balance training)\b',
            r'\b(complex interventions|exergaming)\b',
            r'\b(diagnostic test|procedure)\b',
            r'\b(treatment|therapy|protocol|strategy|experimental group)\b'
        ],
        'Comparison': [
            r'\b(control|comparison|comparator|placebo|standard care|usual care)\b',
            r'\b(trials where the only difference between groups was the exercise intervention)\b',
            r'\b(social contact activities|watchful waiting)\b',
            r'\b(studies only comparing different types of exercise|studies only comparing physical to cognitive exercise)\b',
            r'\b(control group|placebo|reference group|nonintervention)\b'
        ],
        'Outcome': [
            r'\b(outcome|results|endpoints|measures|efficacy|effectiveness)\b',
            r'\b(cognition|activities of daily living|neuropsychiatric symptoms|agitation|aggression|depression|mortality)\b',
            r'\b(caregiver’s burden of care|quality of life|healthcare services costs|physiological outcomes)\b',
            r'\b(brain-derived neurotrophic factor|activity and density of some brain areas)\b',
            r'\b(accurate diagnosis|relieve or improve symptoms|maintain function)\b',
            r'\b(result|effect|assessment|evaluation|primary outcome|secondary outcome)\b'
        ]
    }
    
    # Function to check if text matches any of the patterns
    def match_criteria(text, patterns):
        for pattern in patterns:
            if re.search(pattern, text, re.IGNORECASE):
                return True
        return False
    
    # Chunk text if necessary
    token_chunks = chunk_text(text, tokenizer)
    
    for chunk in token_chunks:
        # Decode the chunk back to text
        chunked_text = tokenizer.decode(chunk[0], skip_special_tokens=True)
        entities = nlp(chunked_text)
        
        for entity in entities:
#             print(entity['word'])
            for component, patterns in criteria_patterns.items():
                if match_criteria(entity['word'], patterns):
                    pico[component] = True
    
    pico['PICO_compliant'] = all(pico.values())
    return pico

In [121]:
pico_text = """
Title:
The Effect of Aerobic Exercise on Cognitive Function in Elderly Patients with Mild Cognitive Impairment

Abstract:
This study investigates the impact of a structured aerobic exercise program on cognitive function in elderly patients diagnosed with mild cognitive impairment (MCI). We conducted a randomized controlled trial to compare the cognitive outcomes of patients participating in the exercise program versus a control group receiving usual care.

Introduction:
Mild cognitive impairment (MCI) is a condition characterized by cognitive decline that is more pronounced than normal aging but not severe enough to interfere significantly with daily activities. Exercise has been proposed as a potential intervention to improve cognitive function in this population.

Methods:
Population: The study included 100 elderly patients aged 65 and above diagnosed with MCI based on clinical criteria.

Intervention: Participants in the intervention group engaged in a structured aerobic exercise program, consisting of 45-minute sessions of moderate-intensity exercise, three times per week for six months.

Comparison: The control group received usual care, which included standard medical treatment and advice on maintaining a healthy lifestyle but did not include a structured exercise program.

Outcome: The primary outcome measure was the change in cognitive function, assessed using the Mini-Mental State Examination (MMSE) score at baseline and after six months. Secondary outcomes included changes in memory, attention, and executive function, assessed using standardized neuropsychological tests.

Results:
At the end of the study period, the intervention group showed a statistically significant improvement in MMSE scores compared to the control group (mean difference: 2.5 points, 95% CI: 1.2-3.8, p < 0.001). Improvements were also observed in memory and executive function tests.

Conclusion:
Aerobic exercise has a positive effect on cognitive function in elderly patients with MCI. Regular participation in structured exercise programs may be a beneficial non-pharmacological intervention to delay cognitive decline in this population.

Keywords: mild cognitive impairment, aerobic exercise, cognitive function, elderly, randomized controlled trial

Discussion:
Our findings suggest that aerobic exercise can significantly improve cognitive function in elderly patients with MCI. This aligns with previous research indicating the benefits of physical activity on brain health. Further studies with larger sample sizes and longer follow-up periods are warranted to confirm these results and explore the mechanisms underlying the observed benefits.

References:

Smith, A., et al. (2021). The effects of physical activity on cognitive function in older adults. Journal of Aging Research, 25(3), 215-229.
Johnson, L., et al. (2020). Exercise interventions for cognitive decline: A systematic review. The Gerontologist, 60(4), 517-529.
"""

In [122]:
result = detect_pico(pico_text)
result

{'Population': False,
 'Intervention': False,
 'Comparison': False,
 'Outcome': False,
 'PICO_compliant': False}

In [153]:
import pandas as pd

papers_df = pd.read_csv("SCIPdb.csv")

In [154]:
import os
from langchain.document_loaders import PyPDFLoader
from tqdm import tqdm

files = papers_df["File Attachments"]
all_data = []
errored_files = []
i = 0
for file in tqdm(files[:6]):
    try:
        file = file.replace("\\", "/").replace(';', '')
        loader = PyPDFLoader(file.strip())
        data = loader.load()
        for d in data:
            d.metadata['Publication Year'] = int(papers_df['Publication Year'][i])
            d.metadata['Date Added'] = int(papers_df['Date Added'][i][:4])
            d.metadata['Author'] = str(papers_df['Author'][i])
            d.metadata['Title'] = str(papers_df['Title'][i])
        all_data.append(data)
    except Exception as e:
        #print(e)
        errored_files.append(file)
    finally:
        i += 1

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.08it/s]


In [171]:
papers = []
for data in all_data:
    text = ""
    for d in data:
        text += d.page_content
    papers.append(text)

In [179]:
results = []
for paper in tqdm(papers):
    result = detect_pico(paper)
    results.append(result)
results

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.40s/it]


[{'Population': True,
  'Intervention': True,
  'Comparison': False,
  'Outcome': True,
  'PICO_compliant': False},
 {'Population': True,
  'Intervention': False,
  'Comparison': False,
  'Outcome': True,
  'PICO_compliant': False},
 {'Population': False,
  'Intervention': False,
  'Comparison': False,
  'Outcome': True,
  'PICO_compliant': False},
 {'Population': False,
  'Intervention': False,
  'Comparison': False,
  'Outcome': False,
  'PICO_compliant': False},
 {'Population': False,
  'Intervention': False,
  'Comparison': False,
  'Outcome': False,
  'PICO_compliant': False},
 {'Population': True,
  'Intervention': False,
  'Comparison': False,
  'Outcome': True,
  'PICO_compliant': False}]